# NEMPY Constraints Analysis

<font color=green>Jupyter Notebook <br>Version: 1.0, Draft Notebook <br>
Author: Declan Heim <br>
Contact: [Email](mailto:d.heim@unsw.edu.au) <br>
GitHub Ref: [nempy_constraints_v1](https://github.com/dec-heim/nempy_constraints_v1)</font>

---

## Introduction & Background

This notebook serves as an example of how constraints can be analysed using the open-source python simulator of AEMO's dispatch process, 'nempy'. This tool was initially developed by Nick Gorman and is expanded upon in this example, with additional functionality to draw insights into the impact of specific constraint equations in historical dispatch. These changes are not yet available via the official nempy release through pip, rather it draws on a forked version of the github repo. Updates to integrate and streamline these features into the official release of nempy are in progress.

The purpose of analysing constraints in this regard is to convey a deeper understanding and discussion around the impact of constraints to those units involved in the constraint. Further, this example demonstrates the outcomes that could be seen in the market by alleviating a specific constraint, both in a market price sense and the volume (MW) dispatched for each unit in the constraint. This type of analysis is possible by re-simulating the same dispatch interval and modifying the inputs that define the historically binding constraint. Such conterfactual work requires the use of a dispatch simulator, unlike higher-level observations which demonstrate the units have been constrained as opposed to what otherwise may have happened.

The example provided assumes the reader has some foundational understanding of the NEM dispatch process, how constraints are defined, the types of constraints and so forth. Hence it can be seen as an intermediate example. Explainations of historical dispatch procedures in nempy are omitted given there is abundant information [covered in this example](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch). To gain a thorough understanding of the work presented here, one should first understand simplier examples of NEM dispatch and have some insight as to how nempy can be used, as provided in [Nempy Documentation.](https://nempy.readthedocs.io/) Some helpful resources to assist in understanding constraints in the NEM are linked below.

### Acknowledgement:
Special thanks to Nick Gorman at UNSW for developing the nempy tool and providing continued support in expanding it's functionality for the constraints analysis here. 

---

### Useful References:
<font color=blue>**Relevant nempy material:**</font>
- [UNSW-CEEM Nempy Github](https://github.com/UNSW-CEEM/nempy)
- [Nempy Documentation](https://nempy.readthedocs.io/)

<font color=blue>**Relevant nempy examples:**</font>
- [Nempy: Simple Examples 1-5](https://nempy.readthedocs.io/en/latest/examples.html#)
- [Nempy: Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch)

<font color=blue>**Relevant material for understanding constraints:**</font>
- [AEMO Constraint Implementation Guidelines (2015)](https://www.aemo.com.au/Electricity/National-Electricity-Market-NEM/Security-and-reliability/-/media/943DDD419E5942B8993CCA8EA201C37E.ashx)
- [AEMO Constraint Naming Guidelines (2013)](https://www.aemo.com.au/-/media/Files/Electricity/NEM/Security_and_Reliability/Congestion-Information/2016/Constraint-Naming-Guidelines.pdf)
- [Example of Marginal Value Calculations of 'X5' constraint, by Allan O'Neil (WattClarity)](https://wattclarity.com.au/articles/2020/12/casestudy-followup-x5-constraint/)

<font color=blue>**Specific material for the constraint that this example draws upon:**</font>
- [Example Analysis of Constraint Impact on PV, by Jack Simpson](https://jacksimpson.co/exploring-the-impact-of-constraints-on-a-solar-farm-in-the-national-electricity-market/)
- [AEMO Feb 2021 Monthly Constraint Report](https://www.aemo.com.au/-/media/files/electricity/nem/security_and_reliability/congestion-information/statistics/2021/monthly-constraint-report-february-2021.pdf?la=en).
---

## Example 1 - Investigating the impact of ____ in Feb 2021
### Event Description


---

## Section A. Model Building
### A1. Import Python Packages


In [28]:
import pandas as pd
!pip install db-sqlite3
import sqlite3

# The forked nempy market dispatch engine is loaded
#import sys
#!{sys.executable} -m pip install git+https://github.com/dec-heim/nempy_constraints_v1.git
!pip install --user git+https://github.com/dec-heim/nempy_constraints_v1.git --no-warn-script-location
from nempy import markets, time_sequential
from nempy.historical_inputs import loaders, mms_db, \
    xml_cache, units, demand, interconnectors, constraints

# Plotting packages
!pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

  Cloning https://github.com/dec-heim/nempy_constraints_v1.git to c:\users\derlu\appdata\local\temp\pip-req-build-_3__e1za
  Resolved https://github.com/dec-heim/nempy_constraints_v1.git to commit 3400720b0375e300d6ea5d2944b4684555d680eb
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'


  Running command git clone -q https://github.com/dec-heim/nempy_constraints_v1.git 'C:\Users\derlu\AppData\Local\Temp\pip-req-build-_3__e1za'


---

### A2. Retrieve + Prepare Historical Data from AEMO

The data for this example has already been extracted and prepared for nempy locally. The data was found for the **15th February 2021** and includes a range of parameters, to name a few:
- unit details
- unit availability
- volume bids
- price bids
- unit ramp rate constraints
- interconnector models with loss equations
- regional demand constraints
- various other constraints such as FCAS, generic constraint sets

Further steps on how to download the data from AEMO MMS and configure this for nempy are provided in the [Detailed Recreation of Historical Dispatch Example](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) within nempy documentation.

In [29]:
con = sqlite3.connect('feb_2021_mms.db')
mms_db_manager = mms_db.DBManager(connection=con)
xml_cache_manager = xml_cache.XMLCacheManager('feb_2021_cache')

raw_inputs_loader = loaders.RawInputsLoader(
    nemde_xml_cache_manager=xml_cache_manager,
    market_management_system_database=mms_db_manager)

---

### A3. Define Dispatch Intervals
For this specific example, we will look at one specific dispatch interval for **15th February 2021** which is the interval at **15:00:00**.

> <font color=red>**IS THIS TIME STARTING OR ENDING??**</font>

Once the interval has been defined, the subsequent functions are called to load the respective data from our previously downloaded database.

In [30]:
dispatch_intervals = ['2021/02/15 15:00:00','2021/02/15 15:05:00']
                      
interval = dispatch_intervals[0]

In [31]:
outputs = []
raw_inputs_loader.set_interval(interval)
unit_inputs = units.UnitData(raw_inputs_loader)
interconnector_inputs = interconnectors.InterconnectorData(raw_inputs_loader)
constraint_inputs = constraints.ConstraintData(raw_inputs_loader)
demand_inputs = demand.DemandData(raw_inputs_loader)


---
### A4. Define Market Data
The first market data that we require are the unit information for each DUID, specifying a region, dispatch type and loss factor. This unit data is used in defining the market object and regions which we want to construct in nempy.

In [32]:
unit_info = unit_inputs.get_unit_info()
market = markets.SpotMarket(market_regions=['QLD1', 'NSW1', 'VIC1',
                                            'SA1', 'TAS1'],
                            unit_info=unit_info)
print(f"\nThe Unit Details for the interval of {interval} are:")
unit_info


The Unit Details for the interval of 2021/02/15 15:00:00 are:


,unit,region,dispatch_type,loss_factor
0,AGLHAL,SA1,generator,0.966600
1,AGLNOW1,NSW1,generator,0.996700
2,AGLSITA1,NSW1,generator,1.001200
3,AGLSOM,VIC1,generator,0.984163
4,ANGAST1,SA1,generator,0.996813
...,...,...,...,...
537,YWNL1,VIC1,generator,0.954900
538,YWPS1,VIC1,generator,0.966000
539,YWPS2,VIC1,generator,0.954900
540,YWPS3,VIC1,generator,0.954900


Once the market object is created we can specify the volume and price bids for each generator and load.

In [33]:
volume_bids, price_bids = unit_inputs.get_processed_bids()

market.set_unit_volume_bids(volume_bids)
market.set_unit_price_bids(price_bids)

print(f"\nThe Volume Bids for the interval of {interval} are:")
volume_bids


The Volume Bids for the interval of 2021/02/15 15:00:00 are:


,unit,service,1,2,3,4,5,6,7,8,9,10
0,AGLHAL,energy,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,195.0
1,AGLSOM,energy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,170.0
2,ANGAST1,energy,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,50.0
6,ARWF1,energy,0.0,241.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,BALBG1,energy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...
407,GANNBL1,lower_reg,5.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
408,GANNBL1,raise_reg,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
411,MPP_2,lower_reg,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,25.0
412,MPP_2,raise_reg,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0


In [34]:
print(f"\nThe Price Bids for the interval of {interval} are:")
price_bids


The Price Bids for the interval of 2021/02/15 15:00:00 are:


,unit,service,1,2,3,4,5,6,7,8,9,10
0,AGLHAL,energy,-966.60000,0.000000,269.497746,356.491746,404.821746,482.149746,559.477746,1319.950296,10225.535742,14499.000000
1,AGLSOM,energy,-984.16308,0.000000,59.049785,83.653862,142.703647,279.502315,456.651669,985.147243,13090.225186,14762.406833
2,ANGAST1,energy,-996.81310,0.000000,124.601637,199.561983,298.236511,378.769042,588.109761,1370.468491,10584.161496,14952.196500
3,ARWF1,energy,-898.30000,-156.995891,2.003209,3.997435,8.003853,15.998723,31.997446,64.003875,127.998767,13025.350000
4,BALBG1,energy,-964.30000,0.000000,48.108927,89.573827,97.288227,126.217227,250.611927,432.864627,9739.439643,14464.500000
...,...,...,...,...,...,...,...,...,...,...,...,...
708,VSSEL1V1,raise_60s,0.00000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,13000.000000
709,VSSEL1V1,raise_6s,0.00000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,8.000000,10.000000,13000.000000
710,VSSSE1V1,raise_5min,0.05000,1.000000,3.000000,5.000000,10.000000,100.000000,300.000000,1000.000000,10000.000000,15000.000000
711,VSSSE1V1,raise_60s,0.05000,1.000000,3.000000,5.000000,10.000000,100.000000,300.000000,1000.000000,10000.000000,15000.000000


The subsequent data loaded into the nempy market object follow the [Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) so we will omit here for brevity and highlight only the generic constraints which are of interest in this analysis.

In [35]:
# Set bid in capacity limits
unit_bid_limit = unit_inputs.get_unit_bid_availability()
market.set_unit_bid_capacity_constraints(unit_bid_limit)
cost = constraint_inputs.get_constraint_violation_prices()['unit_capacity']
market.make_constraints_elastic('unit_bid_capacity', violation_cost=cost)

# Set limits provided by the unconstrained intermittent generation
# forecasts. Primarily for wind and solar.
unit_uigf_limit = unit_inputs.get_unit_uigf_limits()
market.set_unconstrained_intermitent_generation_forecast_constraint(
    unit_uigf_limit)
cost = constraint_inputs.get_constraint_violation_prices()['uigf']
market.make_constraints_elastic('uigf_capacity', violation_cost=cost)

# Set unit ramp rates.
ramp_rates = unit_inputs.get_ramp_rates_used_for_energy_dispatch()
market.set_unit_ramp_up_constraints(
    ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_up_rate']])
market.set_unit_ramp_down_constraints(
    ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_down_rate']])
cost = constraint_inputs.get_constraint_violation_prices()['ramp_rate']
market.make_constraints_elastic('ramp_up', violation_cost=cost)
market.make_constraints_elastic('ramp_down', violation_cost=cost)

# Set unit FCAS trapezium constraints.
unit_inputs.add_fcas_trapezium_constraints()
cost = constraint_inputs.get_constraint_violation_prices()['fcas_max_avail']
fcas_availability = unit_inputs.get_fcas_max_availability()
market.set_fcas_max_availability(fcas_availability)
market.make_constraints_elastic('fcas_max_availability', cost)
cost = constraint_inputs.get_constraint_violation_prices()['fcas_profile']
regulation_trapeziums = unit_inputs.get_fcas_regulation_trapeziums()
market.set_energy_and_regulation_capacity_constraints(regulation_trapeziums)
market.make_constraints_elastic('energy_and_regulation_capacity', cost)
scada_ramp_down_rates = unit_inputs.get_scada_ramp_down_rates_of_lower_reg_units()
market.set_joint_ramping_constraints_lower_reg(scada_ramp_down_rates)
market.make_constraints_elastic('joint_ramping_lower_reg', cost)
scada_ramp_up_rates = unit_inputs.get_scada_ramp_up_rates_of_raise_reg_units()
market.set_joint_ramping_constraints_raise_reg(scada_ramp_up_rates)
market.make_constraints_elastic('joint_ramping_raise_reg', cost)
contingency_trapeziums = unit_inputs.get_contingency_services()
market.set_joint_capacity_constraints(contingency_trapeziums)
market.make_constraints_elastic('joint_capacity', cost)

# Set interconnector definitions, limits and loss models.
interconnectors_definitions = \
    interconnector_inputs.get_interconnector_definitions()
loss_functions, interpolation_break_points = \
    interconnector_inputs.get_interconnector_loss_model()
market.set_interconnectors(interconnectors_definitions)
market.set_interconnector_losses(loss_functions,
                                  interpolation_break_points)

# Add FCAS market constraints.
fcas_requirements = constraint_inputs.get_fcas_requirements()
market.set_fcas_requirements_constraints(fcas_requirements)
violation_costs = constraint_inputs.get_violation_costs()
market.make_constraints_elastic('fcas', violation_cost=violation_costs) 


---

### A5. Relevant Constraints Data
A few key dataframes should be highlighted as part of the [Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) which are directly relevant to the constraint equation we will assess.

<ol>
    <li>The <b>Generic RHS dataframe:</b> This data contains columns specifying the constraint set (more 'correctly' referred to as constraint equation), the RHS value, and the type. In simple terms, the constraint equation defines the name of the constraint, the RHS value is the limit (generally maximum, minimum, or equal to) value which the sum of the LHS arguments must comply with, and the type is the defined mathematically operator (less than and equal, more than and equal, or equal to). </li>
    <li>The <b>Unit Generic LHS dataframe:</b> This dataframe maps each unit assosciated with each constraint equation. The columns specified include the constraint 'set', the unit that is found in the LHS of the constraint equation, the service involved (e.g. energy), and the LHS coefficient assosciated with the unit (effectively the multiplier for the unit dispatch volume in MWs).
    <li>The <b>Interconnector Generic LHS dataframe:</b> This dataframe is akin to the unit generic lhs above however maps the interconnectors found as LHS arguments for specific constraints and their coefficients.    
</ol>

A final note is that violation costs apply to each constraint set here. These costs have been specified in the prior code section with the line <code>violation_costs = constraint_inputs.get_violation_costs()</code> . In essence these generic constraints are 'soft' contraints that could theoretically be violated but the dispatch engine would assosciate (usually an extremely high cost) for each MW the constraint equation is violated. Such a strucutre allows the dispatch engine to violate constraints in a preferred order, e.g. FCAS requirements violated before thermal constraints. 

In [36]:
# Add generic constraints, RHS parameters
generic_rhs = constraint_inputs.get_rhs_and_type_excluding_regional_fcas_constraints()
market.set_generic_constraints(generic_rhs)
market.make_constraints_elastic('generic', violation_cost=violation_costs)

print(f"\nThe Generic Constraints RHS are defined for {interval} as:")
generic_rhs


The Generic Constraints RHS are defined for 2021/02/15 15:00:00 as:


,set,rhs,type
0,#BANGOWF1_E,11.000000,<=
1,#BBTHREE3_E,25.000000,<=
2,#BULGANA1_E,100.000000,<=
3,#COHUNSF1_E,0.000000,<=
4,#COLWF01_E,146.000000,<=
...,...,...,...
822,V_OWF_TGTSNRBHTN_30,10030.000000,<=
823,V_S_HEYWOOD_UFLS,199.655472,<=
824,V_S_NIL_ROCOF,408.000000,<=
825,V_T_NIL_BL1,478.000000,<=


In [37]:
# Add generic constraints, LHS coefficients DUIDs
unit_generic_lhs = constraint_inputs.get_unit_lhs()
market.link_units_to_generic_constraints(unit_generic_lhs)

print(f"\nThe Generic Constraints DUID LHS coefficients are defined for {interval} as:")
unit_generic_lhs


The Generic Constraints DUID LHS coefficients are defined for 2021/02/15 15:00:00 as:


,set,unit,service,coefficient
0,#BANGOWF1_E,BANGOWF1,energy,1.0
1,#BBTHREE3_E,BBTHREE3,energy,1.0
2,#BULGANA1_E,BULGANA1,energy,1.0
3,#COHUNSF1_E,COHUNSF1,energy,1.0
4,#COLWF01_E,COLWF01,energy,1.0
...,...,...,...,...
9379,V_MTGBRAND_44WT,MTGELWF1,energy,1.0
9380,V_MWWF_GFT1_5,MUWAWF1,energy,1.0
9381,V_OAKHILL_TFB_42,OAKLAND1,energy,1.0
9382,V_OWF_NRB_0,OAKLAND1,energy,1.0


In [38]:
# Add generic constraints, LHS coeffients ICs
interconnector_generic_lhs = constraint_inputs.get_interconnector_lhs()
market.link_interconnectors_to_generic_constraints(
    interconnector_generic_lhs)

print(f"\nThe Generic Constraints Interconnector LHS coefficients are defined for {interval} as:")
interconnector_generic_lhs


The Generic Constraints Interconnector LHS coefficients are defined for 2021/02/15 15:00:00 as:


,set,interconnector,coefficient
0,DATASNAP,N-Q-MNSP1,1.0
1,DATASNAP_DFS_LS,N-Q-MNSP1,1.0
2,DATASNAP_DFS_NCAN,N-Q-MNSP1,1.0
3,DATASNAP_DFS_NCWEST,N-Q-MNSP1,1.0
4,DATASNAP_DFS_NNTH,N-Q-MNSP1,1.0
...,...,...,...
672,V^^S_PAVC_MAXG-DS,V-SA,1.0
673,V_S_HEYWOOD_UFLS,V-SA,1.0
674,V_S_NIL_ROCOF,V-SA,1.0
675,V_T_NIL_BL1,T-V-MNSP1,-1.0


---

### A6. Set Demand Requirements and Dispatch the Market
The final stage of this model building section is to define the regional demand requirements and run the market dispatch. Again, see [Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) for additional details regarding the fast start and over constrained rerun.

In [39]:
# Set the operational demand to be met by dispatch.
regional_demand = demand_inputs.get_operational_demand()
market.set_demand_constraints(regional_demand)

print(f"\nThe Demand to be met for {interval} is:")
regional_demand


The Demand to be met for 2021/02/15 15:00:00 is:


,region,demand
0,NSW1,7145.27
1,QLD1,6783.86
2,SA1,811.93
3,TAS1,1065.63
4,VIC1,4494.12


In [40]:
# Get unit dispatch without fast start constraints and use it to
# make fast start unit commitment decisions.
market.dispatch()
dispatch = market.get_unit_dispatch()
fast_start_profiles = unit_inputs.get_fast_start_profiles_for_dispatch(dispatch)
market.set_fast_start_constraints(fast_start_profiles)
if 'fast_start' in market.get_constraint_set_names():
    cost = constraint_inputs.get_constraint_violation_prices()['fast_start']
    market.make_constraints_elastic('fast_start', violation_cost=cost)

# If AEMO historical used the over constrained dispatch rerun
# process then allow it to be used in dispatch. This is needed
# because sometimes the conditions for over constrained dispatch
# are present but the rerun process isn't used.
if constraint_inputs.is_over_constrained_dispatch_rerun():
    market.dispatch(allow_over_constrained_dispatch_re_run=True,
                    energy_market_floor_price=-1000.0,
                    energy_market_ceiling_price=14500.0,
                    fcas_market_ceiling_price=1000.0)
else:
    # The market price ceiling and floor are not needed here
    # because they are only used for the over constrained
    # dispatch rerun process.
    market.dispatch(allow_over_constrained_dispatch_re_run=False)

---
## Section B - Initial Results (considered all historical constraints)
### B1. Market Price Outcomes
The cleared market prices per region for the simulated historical dispatch interval are found below.

In [41]:
prices = market.get_energy_prices()
prices['time'] = interval

print(f"\nThe Cleared Market Prices per region for {interval} are:")
prices.loc[:,['region','price']]


The Cleared Market Prices per region for 2021/02/15 15:00:00 are:


,region,price
0,NSW1,37.192685
1,QLD1,37.730000
2,SA1,9.793487
3,TAS1,34.881000
4,VIC1,9.890000


---

### B2. Identifying the Binding Constraints
<b><font color=blue>This section applies additional functionality of the [modified nempy package](https://github.com/dec-heim/nempy_constraints_v1)</font></b>

The binding constraints from the dispatch interval can be found by calling the function <code>market.get_constraint_marginal_values</code>.
This function searches our previously defined constraint inputs which we defined in the dataframes called <b> Generic RHS</b>, <b>Unit Generic LHS</b>, and <b>Unit Interconnector LHS</b>. For each generic constraint that has been applied to the dispatch process, the marginal value or MV is retrieved from the python mixed-integer-linear-program as a 'byproduct' in a sense of optimising the most economical dispatch.

In simple terms, the MV can be thought of as the amount by which the system dispatch cost would change should the constraint be relaxed by 1 MW. An example might be allowing more powerflow through a network line linking two nodes. Generally, we have a LHS <= RHS structure for thermal constraints. By increasing the RHS by 1MW, we would allow 1MW more power to be transferred through the line. Let's say this is 1MW more power from a cheap solar farm, so we no longer have to run a more expensive gas generator which is located somewhere else in the network. As such, the MV in this case would be negative, signifying that there is a cost reduction on the system dispatch. <b> This is not to be confused with the market clearing price </b>... more on that later. A final note is that if a constraint is not binding, it would have a zero MV. The unit outputs or interconnector flows on the LHS are not being restricted, so it is indifferent to relaxing the RHS by 1MW. 

The concept of marginal values is better explained in the [Example of Marginal Value Calculations of 'X5' constraint, by Allan O'Neil (WattClarity)](https://wattclarity.com.au/articles/2020/12/casestudy-followup-x5-constraint/)


In [44]:
marginal_values = market.get_constraint_marginal_values()
marginal_values

                     set  constraint_id type           rhs         slack  \
0            #BANGOWF1_E           2409   <=     11.000000     11.000000   
1            #BBTHREE3_E           2410   <=     25.000000     25.000000   
2            #BULGANA1_E           2411   <=    100.000000    100.000000   
3            #COHUNSF1_E           2412   <=      0.000000     -0.000000   
4             #COLWF01_E           2413   <=    146.000000     47.180000   
..                   ...            ...  ...           ...           ...   
667  V_OWF_TGTSNRBHTN_30           3076   <=  10030.000000  10029.999000   
668     V_S_HEYWOOD_UFLS           3077   <=    199.655472    248.403505   
669        V_S_NIL_ROCOF           3078   <=    408.000000    456.748033   
670          V_T_NIL_BL1           3079   <=    478.000000     26.008452   
671        V_T_NIL_FCSPS           3080   <=    451.991548      0.000000   

     marginal_value  
0          0.000000  
1          0.000000  
2          0.000000  

,set,constraint_id,type,rhs,slack,marginal_value
24,#MWPS2PV1_E,2433,<=,0.000000,-0.000000e+00,-1.009793e+03
27,#PPCCGT_D_E,2436,=,175.000000,0.000000e+00,1.499021e+04
30,#WARWSF1_E,2439,<=,16.000000,0.000000e+00,-1.037730e+03
34,$BARRON-1,2443,=,31.000000,0.000000e+00,1.496227e+04
35,$BLOWERNG,2444,=,33.000000,0.000000e+00,-1.037193e+03
36,$CALL_B_1,2445,=,350.000000,1.136868e-13,1.496227e+04
37,$YATSF1,2446,=,70.000000,0.000000e+00,5.700000e+06
110,F_T_AUFLS2_R6,2519,<=,25.146998,0.000000e+00,-8.050000e+00
178,N>>N-NIL_94T_947,2587,<=,121.937308,0.000000e+00,-5.113113e+02
214,N>>N-PKWL_94K_2,2623,<=,143.702118,0.000000e+00,-1.037193e+03


The results above produce a dataframe identifying the constraint equations with non-zero marginal values. The columns here include the 'set', constraint id, type, rhs, slack and marginal_value. The only fields of concern to us in this example are the constraint 'set' or name and the fact that it has a non-zero MV indicating it is binding.

In this specific dispatch interval we notice a total of 14 binding constraints. To identify what each of these constraints mean, refer to the [AEMO Constraint Naming Guidelines (2013)](https://www.aemo.com.au/-/media/Files/Electricity/NEM/Security_and_Reliability/Congestion-Information/2016/Constraint-Naming-Guidelines.pdf).

---
### B3. Retrieve information about the 'N>>N-NIL_94T_947' constraint
We will select one binding constraint from this dispatch interval to analyse. In this example the <b>N>>N-NIL_94T_947</b> constraint. From which we can compare the earlier analysis of this occurance in the [Example by J. Simpson](https://jacksimpson.co/exploring-the-impact-of-constraints-on-a-solar-farm-in-the-national-electricity-market/)

Deciphering the naming guidelines this can be interpreted as:
- N: New South Wales
- <string>>>: Thermal overload of a network element</string>
- N-NIL: System Normal
- 94T: The network element being line 94T (Molong to Orange North)
- 947: The 'tripping' element which would cause the concerned line to overload, here being 947 (Wellington to Orange North).
Extracting only the information relevant to this constraint can be done slicing the dataframe as below.

Or concisely described by AEMO as: <b>"Out= Nil, avoid O/L Molong to Orange North (94T) on trip of Wellington to Orange North (947), Feedback"</b> A good time to also mention AEMO's reporting on this specific event occuring during Feb 2021 in it's [Monthly Constraint Report](https://www.aemo.com.au/-/media/files/electricity/nem/security_and_reliability/congestion-information/statistics/2021/monthly-constraint-report-february-2021.pdf?la=en).

Subsequently, the <code>market.get_constraint_mapping()</code> function is useful in collecting all the LHS terms involved in this constraint. Currently this returns a dict format of one dataframe of unit lhs, and one dataframe of interconnector lhs. Given it is a thermal constraint of line 94T, no interconnectors are found in the LHS so the latter return is an empty dataframe. We therefore are interested in the units lhs involved.

In [99]:
constraint_rhs_info = marginal_values[marginal_values['set'] == 'N>>N-NIL_94T_947'].reset_index(drop=True)
constraint_rhs_info

,set,constraint_id,type,rhs,slack,marginal_value
0,N>>N-NIL_94T_947,2587,<=,121.937308,0.0,-511.311327


In [64]:
constraintmap = market.get_constraint_mapping('N>>N-NIL_94T_947')
constraintmap['units']

,set,unit,service,coefficient
2549,N>>N-NIL_94T_947,BERYLSF1,energy,0.1355
2550,N>>N-NIL_94T_947,BODWF1,energy,0.2025
2551,N>>N-NIL_94T_947,GOONSF1,energy,0.5049
2552,N>>N-NIL_94T_947,JEMALNG1,energy,0.4273
2553,N>>N-NIL_94T_947,MANSLR1,energy,0.8803
2554,N>>N-NIL_94T_947,NEVERSF1,energy,0.2379
2555,N>>N-NIL_94T_947,NYNGAN1,energy,0.2379
2556,N>>N-NIL_94T_947,PARSF1,energy,0.5049


Note there are 8 unit DUIDs which are LHS terms involved in this binding constraint for this specific interval. Each of these can be identified as:
- Beryl Solar Farm (BERYLSF1) 
- Bodangora Wind Farm (BODWF1)
- Goonumbla Solar Farm (GOONSF1)
- Jemalong Solar Project (JEMALNG1)
- Manildra Solar Farm (MANSLR1)
- Nevertire Solar Farm (NEVERSF1)
- Nyngan Solar Plant (NYNGAN1)
- Parkes Solar Farm (PARSF1)

We can in fact verify that the constraint is binding by summating each units dispatched volume that contributes to the total LHS arguement. Firstly, let's extract the dispatch values for these units and save this matrix as <code>units_lhs_vals</code>.

In [74]:
units = market.get_unit_dispatch()
units_lhs_vals = units[units['unit'].isin(constraintmap['units']['unit'])]
units_lhs_vals

,unit,service,dispatch
47,BERYLSF1,energy,11.623973
53,BODWF1,energy,37.809000
189,GOONSF1,energy,69.000000
267,JEMALNG1,energy,14.998500
383,MANSLR1,energy,0.000000
434,NEVERSF1,energy,98.440000
437,NYNGAN1,energy,95.818000
444,PARSF1,energy,50.000000


Now we can merge this dataframe containing the dispatch volume (MW) and the earlier dataframe of the lhs coefficient factors.

In [82]:
line94t_mtx = pd.merge(left=units_lhs_vals.loc[:,['unit','dispatch']],\
                       right=constraintmap['units'].loc[:,['unit','coefficient']],on='unit')
line94t_mtx

,unit,dispatch,coefficient
0,BERYLSF1,11.623973,0.1355
1,BODWF1,37.809000,0.2025
2,GOONSF1,69.000000,0.5049
3,JEMALNG1,14.998500,0.4273
4,MANSLR1,0.000000,0.8803
5,NEVERSF1,98.440000,0.2379
6,NYNGAN1,95.818000,0.2379
7,PARSF1,50.000000,0.5049


By multiplying each unit's LHS dispatch value by the LHS coefficient, which we are denoting here as LHS contribution or 'contr', we arrive at the total summation of LHS arguments.

In [83]:
line94t_mtx['lhs_contr'] = line94t_mtx['dispatch']*line94t_mtx['coefficient']
line94t_mtx

,unit,dispatch,coefficient,lhs_contr
0,BERYLSF1,11.623973,0.1355,1.575048
1,BODWF1,37.809000,0.2025,7.656322
2,GOONSF1,69.000000,0.5049,34.838100
3,JEMALNG1,14.998500,0.4273,6.408859
4,MANSLR1,0.000000,0.8803,0.000000
5,NEVERSF1,98.440000,0.2379,23.418876
6,NYNGAN1,95.818000,0.2379,22.795102
7,PARSF1,50.000000,0.5049,25.245000


In [101]:
print(f"\n For the constraint at time: {interval}")
print(f"\n The sum of the LHS is {line94t_mtx['lhs_contr'].sum():.4f}")
print(f"\n The RHS limit is {constraint_rhs_info['rhs'][0]:.4f}")



 For the constraint at time: 2021/02/15 15:00:00

 The sum of the LHS is 121.9373

 The RHS limit is 121.9373


Hence, we have checked these values to be equal as suggested by the non-zero marginal value which indicates the constraint is binding.

---
## Section C - Counterfactual Analysis (removing the constraint)
### C1. Removing the 'N>>N-NIL_94T_947' constraint
Having identified and found this constraint to be binding in our historical simulation of dispatch, this section now explains how the constraint can be removed and reveals the impact this has on our results.